In [65]:
import pandas as pd

In [66]:
## for 0:

def process_zero(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]
    df[['participant_name', 'name']] = df['name'].str.rsplit(' ', 1, expand=True)

    df = df.dropna()
    return df

## for 1:

def process_one(df):
    df = df.copy()
    df.dropna()
    return df[['handicap', 'odds', 'participant_name', 'name']]

## for two and six

def process_two_six(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['name', 'participant_name']] = df['name'].str.split(' - ', expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df.dropna()

    return df

## for three and five

def process_three_five(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name', 'description']]

    df[['participant_name', 'description', 'a']] = df['description'].str.rsplit(' ', n=2, expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df = df.drop(columns=['description', 'a'])
    df = df.dropna()

    return df


## for seven and eight

def process_seven_eight(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['participant_name', 'name', 'a']] = df['name'].str.rsplit(' ', n=2, expand=True)

    df = df.drop(columns=['a'])
    df = df.dropna()

    return df

In [70]:
zero = pd.read_csv("0.csv")
one = pd.read_csv("1.csv")
two = pd.read_csv("2.csv")
three = pd.read_csv("3.csv")
five = pd.read_csv("5.csv")
six = pd.read_csv("6.csv")
seven = pd.read_csv("7.csv")
eight = pd.read_csv("8.csv")

zero = process_zero(zero)
one = process_one(one)
two = process_two_six(two)
three = process_three_five(three)
five = process_three_five(five)
six = process_two_six(six)
seven = process_seven_eight(seven)
eight = process_seven_eight(eight)

datasets = [zero, one, two, three, five, six, seven, eight]

In [88]:
concat = pd.concat(datasets, axis=0)
ud = pd.read_csv("ud.csv")
ud.columns = [ud.columns[0], 'handicap']

In [99]:
final = pd.merge(ud, concat, how='inner', on = ['participant_name', 'handicap'])

In [111]:
## agg
def agg(df):    
    result_df = df.groupby(['participant_name', 'name']).agg({'handicap': 'first', 'odds': 'mean'}).reset_index()
    return result_df


In [114]:
final = pivot(agg(final))
final['ev']= final.apply(lambda row: calculate_odds(row['over_odds'], row['under_odds']), axis = 1)
final = final[final['ev'] >= 4.99]

In [132]:
final

,participant_name,line,over_odds,under_odds,ev
0,Alec Burks,1.5,160.666667,-213.333333,13.96
1,Bojan Bogdanovic,2.5,-28.571429,-51.142857,10.36
4,Tyus Jones,7.5,-130.750000,-56.125000,11.18
